In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import r2_score
import tensorflow as tf

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [3]:
BASE_DIR = "../data"

def load_strat(dist, strat):
    strat = strat + "_123" if strat == "random" else strat
    return pd.read_csv(BASE_DIR + f"/stats/{dist}/{dist}_{strat}.csv")


def load_array(dist):
    return np.load(BASE_DIR + f"/stats/{dist}/{dist}.npy", allow_pickle=True)

In [4]:
DISTS = ['3-20-10-weighted', '3-20-10-uniform', '3-20-4-weighted', '3-20-4-uniform',
         'toric-2-0-5-8', 'toric-4-0-5-8', 'toric-6-0-5-8', 'toric-6-0-10-8']

In [5]:
data = {}
for dist in DISTS:
    X = load_array(dist)
    y = load_strat(dist, "degree").PolynomialAdditions.to_numpy()
    train_size = int(0.8 * len(X))
    valid_size = int(0.1 * len(X))
    X_test = X[train_size+valid_size:]
    y_test = y[train_size+valid_size:]
    data[dist] = (X_test, y_test)

In [6]:
model64 = {dist: tf.keras.models.load_model(BASE_DIR + f"/models/{dist}/64")
           for dist in DISTS}
model128 = {dist: tf.keras.models.load_model(BASE_DIR + f"/models/{dist}/128")
            for dist in DISTS}

In [7]:
scores64 = pd.DataFrame(index=pd.Series(DISTS, name="Train"),
                        columns=pd.Series(DISTS, name="Test"))
for train_dist in DISTS:
    for test_dist in DISTS:
        if train_dist.startswith('toric') and not test_dist.startswith('toric'):
            continue
        if not train_dist.startswith('toric') and test_dist.startswith('toric'):
            continue
        X_test, y_test = data[test_dist]
        if test_dist.startswith('toric'):
            y_pred = [model64[train_dist](x[np.newaxis])[0] for x in X_test]
        else:
            y_pred = model64[train_dist](X_test)
        scores64[test_dist][train_dist] = r2_score(y_test, y_pred)

In [8]:
scores64

Test,3-20-10-weighted,3-20-10-uniform,3-20-4-weighted,3-20-4-uniform,toric-2-0-5-8,toric-4-0-5-8,toric-6-0-5-8,toric-6-0-10-8
Train,,,,,,,,
3-20-10-weighted,0.408424,0.0654011,-3.86641,-6.71322,NaN,NaN,NaN,NaN
3-20-10-uniform,0.215135,0.13521,-0.55148,-1.87248,NaN,NaN,NaN,NaN
3-20-4-weighted,0.100309,-0.0373375,0.367314,0.190465,NaN,NaN,NaN,NaN
3-20-4-uniform,0.0204601,-0.578713,0.25635,0.208994,NaN,NaN,NaN,NaN
toric-2-0-5-8,NaN,NaN,NaN,NaN,0.38846,-0.0625136,-21.9802,-0.4281
toric-4-0-5-8,NaN,NaN,NaN,NaN,-2.51329,0.137677,-53.4511,-3.34229
toric-6-0-5-8,NaN,NaN,NaN,NaN,-2.29828,0.00644171,0.712146,0.0592942
toric-6-0-10-8,NaN,NaN,NaN,NaN,-3.19962,-0.0878039,0.0298162,0.453502


In [9]:
scores128 = pd.DataFrame(index=pd.Series(DISTS, name="Train"),
                         columns=pd.Series(DISTS, name="Test"))
for train_dist in DISTS:
    for test_dist in DISTS:
        if train_dist.startswith('toric') and not test_dist.startswith('toric'):
            continue
        if not train_dist.startswith('toric') and test_dist.startswith('toric'):
            continue
        X_test, y_test = data[test_dist]
        if test_dist.startswith('toric'):
            y_pred = [model128[train_dist](x[np.newaxis])[0] for x in X_test]
        else:
            y_pred = model128[train_dist](X_test)
        scores128[test_dist][train_dist] = r2_score(y_test, y_pred)

In [10]:
scores128

Test,3-20-10-weighted,3-20-10-uniform,3-20-4-weighted,3-20-4-uniform,toric-2-0-5-8,toric-4-0-5-8,toric-6-0-5-8,toric-6-0-10-8
Train,,,,,,,,
3-20-10-weighted,0.410009,0.0606835,-4.58104,-6.35702,NaN,NaN,NaN,NaN
3-20-10-uniform,0.230389,0.131703,-5.06466,-6.6195,NaN,NaN,NaN,NaN
3-20-4-weighted,0.0581719,-0.628018,0.366366,0.19446,NaN,NaN,NaN,NaN
3-20-4-uniform,0.0274158,-0.363765,0.285433,0.221953,NaN,NaN,NaN,NaN
toric-2-0-5-8,NaN,NaN,NaN,NaN,0.385458,-0.0424099,-15.4282,-0.322293
toric-4-0-5-8,NaN,NaN,NaN,NaN,-2.5549,0.148482,-78.1974,-3.03474
toric-6-0-5-8,NaN,NaN,NaN,NaN,-2.62671,-0.00230502,0.651573,0.0427453
toric-6-0-10-8,NaN,NaN,NaN,NaN,-3.306,-0.0659894,0.0493067,0.523817
